In [2]:
import healpy as hp
import numpy as np
import random as rand
import statistics as s
import burstutils as bf

This is my workspace where I'm experimenting with the class functions before I end up transferring it into the py version. Some of the things I still need to do while here is:


Documentation

Random gauss vs poisson

Importing modules 

 adding skymaps. 
 
 
 Updating how classes are defined (input really best approach?) 

After redefining classes, build up BurstCube class so that it runs the entire sim! call on module that contains all the functions, length, angle, sep, etc. Should be easiest this way

In [3]:
class GRBs():
   #this produces an error, deal with soon!
    """
    Generates an array of GRB's given 
    certains strength at different sky positions.
    
    Output should be an array. 
    """
    
  #  import numpy as np
   # import healpy as hp
    def __init__(self,NSIDE,strength):
        from healpy import nside2npix
        from healpy import pix2ang
        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(pix2ang(NSIDE,i))

    def say_Ao(self):
        print("The GRBs being tested will be " + str(self.Ao) + " counts strong.")
        
            

In [4]:
GRB = GRBs(4,500)

In [5]:
GRB.pixels

192

In [ ]:
GRB.say_Ao()

In [6]:
class BurstCube():

    def __init__(self,background):

        self.zenith = [0 , 0]
        self.bg = background
        test = input("Are the detectors alternating? ")
        if test == "yes" or test == "Yes" or test == "y" or test == "asdlfkjawe":           
            self.tiltA = np.deg2rad(float(input("Please enter the first tilt (deg) ")))
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
            self.tiltC = self.tiltA
            self.tiltD = self.tiltB
        else:
            self.tilt = np.deg2rad(float(input("Please enter the tilt (deg) ")))
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
            
    
    #make the normal vectors!
        #self.detA = [ zenith[0] + self.tiltA , zenith[1] ] 
        #self.detB = [ zenith[0] + self.tiltB , zenith[1] + np.pi/2 ] 
       # self.detC = [ zenith[0] + self.tiltC , zenith[1] + np.pi ] 
        #self.detD = [ zenith[0] + self.tiltD , zenith[1] + 3*np.pi/2 ] 

        #self.Anorm = hp.ang2vec(self.detA[0],self.detA[1])
        #self.Bnorm = hp.ang2vec(self.detB[0],self.detB[1])
        #self.Cnorm = hp.ang2vec(self.detC[0],self.detC[1])
        #self.Dnorm = hp.ang2vec(self.detD[0],self.detD[1])

    
       # self.dets = [self.Anorm,self.Bnorm,self.Cnorm,self.Dnorm] 
        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB):   #is this how I inherit? 
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        
        self.localizationerrors = []    
        for i in range(len(GRB.sourceangs)):
            sourceAng = GRB.sourceangs[i]
            print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<1:
                detcounts = []  #number of counts incident in each detector. 
                for j in range(len(self.dets)):
                    sep = (bf.angle(sourcexyz,self.dets[j]))
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
                   
                #if sep < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheory=GRB.Ao*bf.response(sep)  #still need to define strength, brb and gonna do that 
                #else: #like I was saying, has to face it!
                    #dtheory = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                    counts = dtheory + self.bg #another artifact, incl this background effect somewhere
                    unccounts = np.sqrt(counts)
                    detactual = rand.gauss(counts,unccounts)  #there is a lot of noise, present, updating it now. 
                    if detactual-self.bg < 0:
                        detactual = 0
                    
                    detcounts.append(detactual)
                
                
                finethetaloc,finephiloc,fineAo = bf.solver(detcounts,self.dets,self.bg)
                
                if finethetaloc > 180:
                #    print("it recovered an unrealistic answer, skip")
                    break
                elif finethetaloc < 0:
                    #print("Same issue, there are limits to theta that are broken here. ")
                    break 
                    
                recpos = [finethetaloc,finephiloc]
              #  print("Recovered position at " + str(recpos))
                
                recvec = hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc))
                locunc.append(bf.angle(sourcexyz,recvec))
               # print("loc unc: " + str(np.rad2deg(bf.angle(sourcexyz,recvec))) + " deg")
                loop+=1
          #  print("Obtained avg loc unc of " + np.rad2deg(s.mean(locunc)) + "")
            if len(locunc)>0:
                self.localizationerrors.append(np.rad2deg(s.mean(locunc)))
            else:
                self.localizationerrors(1000)
                #some big # signifying that it can't catch it 

            print("Loc unc " +  str(np.rad2deg(s.mean(locunc))))
                #loc unc is the uncertainty at each sky position, localerros is for all of them
            
            #should there be more selfs? more prints? basically done so time to debug after lunch. 
        
        maybe = input("All done with the simulation, want to make the skymap too? ")
        
        if maybe == "yes" or maybe == "Yes" or maybe == "y": 
            import matplotlib.pyplot as plt
            from healpy import newvisufunc
            if len(self.localizationerrors) == len(GRB.sourceangs):
                im = np.array(self.localizationerrors)
            else:
                blockedpart=1000*np.ones(GRB.pixels-len(self.localizadtionerrors))
                im = np.concatenate((self.localizationerrors,blockedpart))
            hp.newvisufunc.mollview(im,min=0, max=30,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True,cmap='viridis_r')
            plt.title('All Sky Localization Uncertainty for BurstCube')
        else:
            print("Ok, maybe next time!")

        return self.localizationerrors
            
    
    

In [7]:
cube1 = BurstCube(background=1000)


Are the detectors alternating? 
Please enter the tilt (deg) 45


In [ ]:
cube1.response2GRB(GRB)

Testing [ 11.71585239  45.        ]
Testing detector 0
chi terms: 
[  8.46473700e+01   7.67593768e+01   6.89498520e+01   6.13351266e+01
   5.40431030e+01   4.72005766e+01   4.09218666e+01   3.52998987e+01
   3.04006535e+01   2.62615654e+01   2.28940464e+01   2.02898598e+01
   1.84306165e+01   1.72992699e+01   1.68921700e+01   1.72300685e+01
   1.83664576e+01   2.03918252e+01   2.34328610e+01   2.76464475e+01
   3.32097346e+01   4.03107906e+01   4.91536874e+01   6.00320774e+01
   7.38523258e+01   7.46320490e+08   7.46320490e+08   7.46320490e+08
   7.46320490e+08   7.46320490e+08   7.46320490e+08   7.46320490e+08
   6.83192442e+01   4.86335578e+01   3.18165537e+01   1.79037178e+01
   7.56620843e+00   1.47589655e+00   1.25821268e-01   3.73753890e+00
   1.22058561e+01   2.50718657e+01   4.15233566e+01   6.04230358e+01
   8.03643847e+01   9.97536596e+01   1.16915019e+02   1.30214293e+02
   1.38195696e+02   1.39725039e+02]
Testing detector 1
chi terms: 
[  2.68294123e+02   2.49672002e+02   2

Testing detector 3
chi terms: 
[  3.03967834e+02   2.78411895e+02   2.54906064e+02   2.33548860e+02
   2.14386523e+02   1.97409409e+02   1.82550483e+02   1.69686234e+02
   1.58640118e+02   1.49188276e+02   1.41066887e+02   1.33980471e+02
   1.27610682e+02   1.21625945e+02   1.15693263e+02   1.09494417e+02
   1.02748712e+02   9.52428509e+01   8.68652633e+01   7.76387181e+01
   6.77438286e+01   5.75290335e+01   4.75105118e+01   3.83845707e+01
   3.11933401e+01   9.21313255e+08   9.21313248e+08   9.21313243e+08
   9.21313243e+08   9.21313249e+08   9.21313265e+08   1.75921873e+09
   8.37905501e+08   8.37905505e+08   8.37905516e+08   8.37905534e+08
   8.37905561e+08   8.37905600e+08   8.37905656e+08   8.37905739e+08
   1.52589590e+09   1.52589594e+09   1.52589597e+09   1.52589600e+09
   1.52589604e+09   1.52589607e+09   1.52589610e+09   1.52589614e+09
   1.52589619e+09   6.87990828e+08]
Theta loc 0.0
Loc unc 35.6590876961
Testing [  35.6590877  255.       ]
Testing detector 0
chi terms: 
[ 

/Users/noahkasmanoff/BurstCube/Simulation/Localization_and_Detection/burstutils.py:117: RuntimeWarning: divide by zero encountered in true_divide
  chiterms = chiterms + np.divide((chiresponse-detresponse)**2,detresponse)



Loc unc 60.0
Testing [  60.  135.]
Testing detector 0
chi terms: 
[  7.43250969e+00   5.01764549e+00   3.00431169e+00   1.46592811e+00
   4.61010690e-01   2.13882317e-02   1.42695981e-01   7.78388307e-01
   1.83825298e+00   3.19201463e+00   4.67811925e+00   6.11724202e+00
   7.32951085e+00   8.15394477e+00   8.46822433e+00   8.20668742e+00
   7.37441727e+00   6.05548473e+00   4.41384508e+00   2.68609756e+00
   1.16639432e+00   1.85618564e-01   9.12266180e-02   1.25025571e+00
   4.22114134e+00   9.15425281e+08   9.15425281e+08   9.15425281e+08
   9.15425281e+08   9.15425281e+08   9.15425281e+08   9.15425281e+08
   2.86000332e+00   6.80459969e-02   1.47688472e+00   7.70476809e+00
   1.94933760e+01   3.74010464e+01   6.16780648e+01   9.21865207e+01
   1.28349330e+02   1.69128369e+02   2.13034052e+02   2.58168102e+02
   3.02299698e+02   3.42973257e+02   3.77644004e+02   4.03835643e+02
   4.19312739e+02   4.22259417e+02]
Testing detector 1
chi terms: 
[  1.14538334e+02   1.03556627e+02   9

Loc unc 70.5287793655
Testing [  70.52877937  326.25      ]
Testing detector 0
chi terms: 
[  1.10609242e+02   1.01745075e+02   9.29017565e+01   8.42032698e+01
   7.57898937e+01   6.78053613e+01   6.03851601e+01   5.36470916e+01
   4.76849944e+01   4.25662152e+01   3.83330161e+01   3.50076748e+01
   3.26006044e+01   3.11204296e+01   3.05846662e+01   3.10294786e+01
   3.25169928e+01   3.51388415e+01   3.90150676e+01   4.42883356e+01
   5.11149271e+01   5.96574385e+01   7.00943678e+01   8.27062788e+01
   9.84615925e+01   7.16159394e+08   7.16159394e+08   7.16159394e+08
   7.16159394e+08   7.16159394e+08   7.16159394e+08   7.16159394e+08
   9.21844135e+01   6.94856037e+01   4.94178837e+01   3.19127994e+01
   1.76289309e+01   7.25839873e+00   1.33753892e+00   1.52021388e-01
   3.68081806e+00   1.15683293e+01   2.31231589e+01   3.73437412e+01
   5.29705779e+01   6.85636411e+01   8.26020695e+01   9.36018931e+01
   1.00246383e+02   1.01522934e+02]
Testing detector 1
chi terms: 
[  1.21445893e

[  2.22666639e+02   2.00566008e+02   1.80197315e+02   1.61580845e+02
   1.44720413e+02   1.29604753e+02   1.16208651e+02   1.04494043e+02
   9.44113361e+01   8.59010286e+01   7.88955118e+01   7.33207763e+01
   6.90978949e+01   6.61446475e+01   6.43784297e+01   6.37223018e+01
   6.41160760e+01   6.55330390e+01   6.80001771e+01   7.16167833e+01
   7.65653358e+01   8.31113409e+01   9.15961119e+01   1.02444019e+02
   1.16313079e+02   9.30492914e+08   9.30492930e+08   9.30492947e+08
   9.30492969e+08   9.30492995e+08   9.30493031e+08   1.64749566e+09
   7.17002870e+08   7.17002864e+08   7.17002859e+08   7.17002856e+08
   7.17002858e+08   7.17002866e+08   7.17002885e+08   7.17002923e+08
   1.59017226e+09   1.59017229e+09   1.59017232e+09   1.59017235e+09
   1.59017238e+09   1.59017241e+09   1.59017243e+09   1.59017246e+09
   1.59017249e+09   8.73169963e+08]
Theta loc 0.0
Loc unc 90.0
Testing [  90.    101.25]
Testing detector 0
chi terms: 
[  1.17847159e+01   8.71448552e+00   6.00710125e+00 

Testing detector 1
chi terms: 
[  1.82412719e+02   1.66936755e+02   1.51748134e+02   1.36927932e+02
   1.22613521e+02   1.08986168e+02   9.62560992e+01   8.46459592e+01
   7.43737539e+01   6.56364730e+01   5.85956151e+01   5.33657657e+01
   5.00071965e+01   4.85231817e+01   4.88623800e+01   5.09262399e+01
   5.45810044e+01   5.96735747e+01   6.60503348e+01   7.35782314e+01
   8.21683852e+01   9.18056654e+01   1.02598472e+02   1.14911470e+02
   1.30041491e+02   6.88358273e+08   6.88358271e+08   6.88358271e+08
   6.88358274e+08   6.88358282e+08   6.88358301e+08   1.49835462e+09
   8.09996467e+08   8.09996441e+08   8.09996418e+08   8.09996397e+08
   8.09996379e+08   8.09996365e+08   8.09996355e+08   8.09996349e+08
   8.09996348e+08   8.09996352e+08   8.09996359e+08   8.09996369e+08
   8.09996380e+08   8.09996392e+08   8.09996403e+08   8.09996412e+08
   8.09996418e+08   1.14182975e+02]
Testing detector 2
chi terms: 
[  2.81125767e+02   2.57601415e+02   2.35084048e+02   2.13573445e+02
   1.

Testing detector 2
chi terms: 
[  2.82386009e+02   2.59322520e+02   2.37522388e+02   2.16929047e+02
   1.97488961e+02   1.79150665e+02   1.61863265e+02   1.45575539e+02
   1.30236615e+02   1.15798726e+02   1.02221998e+02   8.94805918e+01
   7.75692327e+01   6.65090575e+01   5.63519074e+01   4.71824865e+01
   3.91181330e+01   3.23062686e+01   2.69199481e+01   2.31524346e+01
   2.12126910e+01   2.13259836e+01   2.37509992e+01   2.88563614e+01
   3.75544015e+01   8.06277280e+08   8.06277281e+08   8.06277286e+08
   8.06277293e+08   8.06277307e+08   8.06277331e+08   1.57470011e+09
   7.68422866e+08   7.68422853e+08   7.68422845e+08   7.68422844e+08
   7.68422853e+08   7.68422875e+08   7.68422916e+08   7.68422988e+08
   1.43163283e+09   1.43163285e+09   1.43163288e+09   1.43163291e+09
   1.43163294e+09   1.43163296e+09   1.43163299e+09   1.43163300e+09
   1.43163301e+09   6.63210252e+08]
Testing detector 3
chi terms: 
[  3.29153091e+02   3.00218789e+02   2.73052144e+02   2.47671100e+02
   2.

[  1.24889457e+02   1.13560710e+02   1.03107630e+02   9.36041879e+01
   8.51895699e+01   7.80593857e+01   7.24527989e+01   6.86368201e+01
   6.68891738e+01   6.74810859e+01   7.06610651e+01   7.66403254e+01
   8.55800579e+01   9.75804192e+01   1.12670940e+02   1.30802092e+02
   1.51837958e+02   1.75550289e+02   2.01614698e+02   2.29610403e+02
   2.59026194e+02   2.89278337e+02   3.19756173e+02   3.49956416e+02
   3.80140838e+02   7.07361661e+08   7.07361666e+08   7.07361668e+08
   7.07361665e+08   7.07361659e+08   7.07361653e+08   1.63285878e+09
   9.25497479e+08   9.25497427e+08   9.25497372e+08   9.25497316e+08
   9.25497260e+08   9.25497209e+08   9.25497167e+08   9.25497141e+08
   1.91172620e+09   1.91172621e+09   1.91172622e+09   1.91172623e+09
   1.91172625e+09   1.91172626e+09   1.91172628e+09   1.91172629e+09
   1.91172629e+09   9.86229160e+08]
Testing detector 3
chi terms: 
[  2.16428059e+02   1.97146652e+02   1.79257541e+02   1.62954584e+02
   1.48459422e+02   1.36013509e+02  

Theta loc 0.0
Loc unc 144.340912304
Testing [ 156.44353569   22.5       ]
Testing detector 0
chi terms: 
[  4.27827554e+01   3.70026824e+01   3.14255018e+01   2.61510178e+01
   2.12804095e+01   1.69038966e+01   1.30900886e+01   9.87819511e+00
   7.27404067e+00   5.25046794e+00   3.75227292e+00   2.70532591e+00
   2.02904442e+00   1.65094684e+00   1.52167891e+00   1.62871097e+00
   2.00689091e+00   2.74423522e+00   3.98178604e+00   5.90712794e+00
   8.74247307e+00   1.27308935e+01   1.81316841e+01   2.52667608e+01
   3.49084871e+01   8.11449474e+08   8.11449474e+08   8.11449474e+08
   8.11449474e+08   8.11449474e+08   8.11449474e+08   8.11449474e+08
   3.09821873e+01   1.78029494e+01   8.00544391e+00   1.84960193e+00
   3.37423529e-02   3.18601009e+00   1.17059502e+01   2.56766739e+01
   4.48112439e+01   6.84276739e+01   9.54528688e+01   1.24456450e+02
   1.53714427e+02   1.81301128e+02   2.05206051e+02   2.23470689e+02
   2.34338982e+02   2.36414197e+02]
Testing detector 1
chi terms: 


In [ ]:
    def plot_skymap(self,GRB):
        import numpy as np
        import matplotlib.pyplot as plt
        import healpy as hp
        from healpy import newvisufunc
        if len(self.localizationerrors) == len(GRB.sourceangs):  #if the function successfully was able to catch all the spots (meaning nothing went wrong!)
            im = np.array(self.localizationerrors) 
        else:
            blockedpart=1000*np.ones(GRB.pixels-len(angoffset))
            im=np.concatenate((self.localizationerrors,blockedpart))
        hp.newvisufunc.mollview(im,min=0, max=50,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True,cmap='viridis_r')
        plt.title('All Sky Localization Uncertainty for BurstCube')
        

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from healpy import newvisufunc
%matplotlib notebook
if len(im) == 48:
    im = np.array(im)
else:
    blockedpart= 1000*np.ones(48-len(im))
    im=np.concatenate((im,blockedpart))
hp.newvisufunc.mollview(im,min=0, max=50,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True,cmap='viridis_r')


In [ ]:
im = [12.193725149835284,
 25.25448240996645,
 22.305707106859245,
 23.965693073949609,
 52.008548584668084,
 27.821629706579458,
 37.423685868114056,
 68.476263273719425,
 30.625361189430574,
 23.401815051293706,
 43.663123284390203,
 19.564729694837329,
 51.433859688574401,
 74.844783683652736,
 56.576057682336121,
 33.323870897313476,
 90.612650101315353,
 82.253008561973189,
 80.329406453087046,
 94.19026553811679,
 99.538349117642525,
 75.175862782099529,
 51.048488219348087,
 74.128524449751453,
 58.480977383443751,
 52.597647136484362,
 77.222734640119555,
 103.61494510174424,
 41.109554839392487,
 76.269478377179141,
 98.202392939060829,
 99.73653357751094,
 122.38781518900511,
 74.733648939874953,
 99.483079370711494,
 100.81648250231294,
 142.5793551632658,
 128.99519400895949,
 109.03771544592294,
 98.612082515419004,
 127.9138150277825,
 93.497546429808551,
 141.23279694489702,
 130.06560240843098,
 157.61167766859899,
 129.80910207180099]


In [ ]:
chiterms = np.zeros(1000)

In [ ]:
chiterms = chiterms + np.linspace(0,999,1000)

In [ ]:
chiterms += np.linspace(0,999,1000)

In [ ]:
len(chiterms)

In [ ]:
cube1.response2GRB(GRB)

In [ ]:
cube1.plot_skymap()

In [ ]:
import matplotlib.pyplot as plt
plt.show()

In [ ]:
#to generate an array of all the source angles, and basically follow that 

GRB = GRBs(2)  #only thing I have to identify is the nside
len(GRB.sourceangs)

In [ ]:
import numpy as np
import healpy as hp
import burstfuncs as bf
import random as rand

class GRBs():
 
    """
    Generates an array of GRB's with some given strength at different sky positions.
    
    Output should be an array. 
    """
    def __init__(self,NSIDE,strength):
        import numpy as np
        import healpy as hp
        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = hp.nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(hp.pix2ang(NSIDE,i))


class BurstCube:
    def __init__(self,background):
        import numpy as np
        import healpy as hp
        import burstfuncs as bf
        self.zenith = [0 , 0]
        self.bg = background
        test = input("Are the detectors alternating? ")
        if test == "yes" or test == "Yes" or test == "y" or test == "asdlfkjawe":           
            self.tiltA = np.deg2rad(float(input("Please enter the first tilt (deg) ")))
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
            self.tiltC = self.tiltA
            self.tiltD = self.tiltB
        else:
            self.tilt = np.deg2rad(float(input("Please enter the tilt (deg) ")))
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
            
        
    @property
    def detA(self):
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        return [ self.zenith[0] + self.tiltC , self.zenith[1] ]
    @property 
    def detD(self):
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    
    def respond2GRB(self, GRB):   #is this how I inherit? 
        #first need to include the GRB.
        
        #Putting these here as a reminder, need to learn how to implement Gaussian fluctuations, hopefully in a similar to the actions gauss(x,sigmax)
        #Gauss = True
        #Poisson = False


        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        localizationerrors = []    
        for i in range(len(GRB.sourceangs)):
            sourceAng = GRB.sourceangs[i]
           # print("Source angle is " + str(sourceAng))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<3:
                detcounts = []  #number of counts incident in each detector. 
                for j in range(len(self.dets)):
                    sep=bf.angle(sourcexyz,self.dets[j])
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                    if sep < np.pi/2: # meaning if >90, would not be facing detector.
                        dtheory=GRB.Ao*bf.response(sourcexyz,self.dets[j])  #still need to define strength, brb and gonna do that 
                    else: #like I was saying, has to face it!
                        dtheory = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                    counts = dtheory + self.bg #another artifact, incl this background effect somewhere
                    unccounts = np.sqrt(counts)
                    detactual = rand.gauss(counts,unccounts)  #there is a lot of noise, present, updating it now. 
                    if detactual-self.bg < 0:
                        detactual = 0
                    
                    detcounts.append(detactual)
    
                   #now for the LSF method using chi squared, applies a coarse to fine refactor here. 
                   #plenty of old variable names to update as well. 
                
                   # print("after fluctuations, " + str(detactual))
                    #fixed indentation errors, uninstalled module, legacy code all fixed. Passes. 

                
                
                coarsethetaloc,coarsephiloc,coarseAo = bf.solver(detcounts,self.dets,0,90,0,360,12,self.bg)
                finethetaloc,finephiloc,fineAo = bf.solver(detcounts,self.dets,coarsethetaloc-5,coarsethetaloc+5,coarsephiloc-7,coarsephiloc+7,12,self.bg)
                
                if finethetaloc > 180:
                #    print("it recovered an unrealistic answer, skip")
                    break
                elif finethetaloc < 0:
                    #print("Same issue, there are limits to theta that are broken here. ")
                    break 
                    
                recpos = [finethetaloc,finephiloc]
                recvec = hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc))
                locunc.append(bf.angle(sourcexyz,recvec))
               # print("loc unc" + str(locunc))
                loop+=1
            print(np.rad2deg(s.mean(locunc)))

            localizationerrors.append(np.rad2deg(s.mean(locunc)))

           # print("obtained error of" +  str(s.mean(locunc)))
                #loc unc is the uncertainty at each sky position, localerros is for all of them
            
            #should there be more selfs? more prints? basically done so time to debug after lunch. 
        return localizationerrors
        

In [ ]:
GRB = GRBs(NSIDE=8,strength=500)

In [ ]:
cubesat = BurstCube(background=1000)

In [ ]:
cubesat.bg

In [ ]:
cubesat.respond2GRB(GRB)

In [ ]:
import numpy as np

In [ ]:
type(np.arange(0,100))


In [ ]:
type(np.linspace(0,100,101))

In [ ]:
        
        Chisep = 


        
        for sa in range(len(oa)): 
            for sb in range(len(ob)):
                for sc in range(len(Aofit)):
                    #make sure it fits within the acceptable range
                    if oa[sa]>=0 and oa[sa]<=np.pi and ob[sb]>=0 and ob[sb]<=2*np.pi:
                        CHIsourceang=[oa[sa],ob[sb]]
                        CHIsourcexyz = hp.ang2vec(CHIsourceang[0],CHIsourceang[1])
                        CHIsep=angle(CHIsourcexyz,detnorms[s])                                           
                        if CHIsep<np.pi/2: 
                            chi=Aofit[sc]*response(CHIsourcexyz,detnorms[s])+bgrd
                            #print("Chi test angle"+str(CHIsourcexyz))
                            #print("detector"+str(dets[s]))
                            #print("chi sometiems"+str(chi))
                            #print("separation here, is it okay? " +str(np.rad2deg(CHIsep)))

                            #this produces nan error, se
                            
                        else:
                            chi=0            
                        if detsvals[s]>0:   #if there is a signal in the detector 
                            chiterm=((chi-detsvals[s])**2/detsvals[s])
                        else:    #if not, just zero 
                            chiterm=1e10

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
SkyCoord.separation()